In [ ]:
pip install bayesian-optimization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from bayes_opt import BayesianOptimization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import os 
from os.path import join
import tqdm
from glob import glob
import tensorflow as tf

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import grey2rgb

from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, AveragePooling2D, MaxPooling2D, Conv2D, SeparableConv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50

from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, \
    confusion_matrix, precision_score, recall_score, f1_score

import PIL
import random
import cv2 as cv
import seaborn as sns

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
IMAGE_SIZE = [224, 224]
EPOCHS = 50

In [ ]:
data_folder = "/content/drive/MyDrive/MS-Upgrad/alzheimers-1"

## images NonDemented
df_NonDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "NonDemented", "*"))),
    "y": 0,
    "class": "NonDementia"
})
shuffled_non = df_NonDemented_imgs.sample(frac=1)
testsize_non = int(0.2 * len(shuffled_non))
# Test
non_test = shuffled_non[:testsize_non]
# Train and validation
non_trainval = shuffled_non[testsize_non:]
trainsize_non = int(0.8 * len(non_trainval))
# Train
shuffled_non_train = non_trainval[:trainsize_non]
#Validation
shuffled_non_val = non_trainval[trainsize_non:]


## images VeryMildDemented
df_VeryMildDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "VeryMildDemented", "*"))),
    "y": 1,
    "class": "VeryMildDementia"
})
shuffled_verymild = df_VeryMildDemented_imgs.sample(frac=1)
testsize_verymild = int(0.2 * len(shuffled_verymild))

# Test
verymild_test = shuffled_verymild[:testsize_verymild]

# Train and validation
verymild_trainval = shuffled_verymild[testsize_verymild:]
trainsize_verymild = int(0.8 * len(verymild_trainval))

# Train
shuffled_verymild_train = verymild_trainval[:trainsize_verymild]

#Validation
shuffled_verymild_val = verymild_trainval[trainsize_verymild:]


## images MildDemented
df_MildDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "MildDemented", "*"))),
    "y": 2,
    "class": "MildDementia"
})
shuffled_mild = df_MildDemented_imgs.sample(frac=1)
testsize_mild = int(0.2 * len(shuffled_mild))

# Test
mild_test = shuffled_mild[:testsize_mild]

# Train and validation
mild_trainval = shuffled_mild[testsize_mild:]
trainsize_mild = int(0.8 * len(mild_trainval))

# Train
shuffled_mild_train = mild_trainval[:trainsize_mild]

#Validation
shuffled_mild_val = mild_trainval[trainsize_mild:]

## images ModerateDemented
df_ModerateDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "ModerateDemented", "*"))),
    "y": 3,
    "class": "ModerateDementia"
})
shuffled_moderate = df_ModerateDemented_imgs.sample(frac=1)
testsize_moderate = int(0.2 * len(shuffled_moderate))

# Test
moderate_test = shuffled_moderate[:testsize_moderate]

# Train and validation
moderate_trainval = shuffled_moderate[testsize_moderate:]
trainsize_moderate = int(0.8 * len(moderate_trainval))

# Train
shuffled_moderate_train = moderate_trainval[:trainsize_moderate]

#Validation
shuffled_moderate_val = moderate_trainval[trainsize_moderate:]


## Number of images
print("TOTAL:")
print("# of images with NonDemented Alzheimer =", len(shuffled_non))
print("# of images with VeryMildDemented Alzheimer =", len(shuffled_verymild))
print("# of images with MildDemented Alzheimer =", len(shuffled_mild))
print("# of images with ModerateDemented Alzheimer =", len(shuffled_moderate))
print("------------")
print("\nTest:")
print("# of images with NonDemented Alzheimer =", len(non_test))
print("# of images with VeryMildDemented Alzheimer =", len(verymild_test))
print("# of images with MildDemented Alzheimer =", len(mild_test))
print("# of images with ModerateDemented Alzheimer =", len(moderate_test))
print("------------")
print("\nTraining:")
print("# of images with NonDemented Alzheimer =", len(shuffled_non_train))
print("# of images with VeryMildDemented Alzheimer =", len(shuffled_verymild_train))
print("# of images with MildDemented Alzheimer =", len(shuffled_mild_train))
print("# of images with ModerateDemented Alzheimer =", len(shuffled_moderate_train))
print("------------")
print("\nValidation:")
print("# of images with NonDemented Alzheimer =", len(shuffled_non_val))
print("# of images with VeryMildDemented Alzheimer =", len(shuffled_verymild_val))
print("# of images with MildDemented Alzheimer =", len(shuffled_mild_val))
print("# of images with ModerateDemented Alzheimer =", len(shuffled_moderate_val))

In [ ]:
# show class imbalance in the dataset

heights = [len(shuffled_non), len(shuffled_verymild), len(shuffled_mild), len(shuffled_moderate)]

fig, ax = plt.subplots()
height = heights
bars = ('Non', 'Very Mild', 'Mild', 'Moderate')
y_pos = np.arange(len(bars))
plt.bar(y_pos, height)
plt.xticks(y_pos, bars)
plt.show()

In [ ]:
def load_img(fname):
    img = cv.imread(fname)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    resize = (IMAGE_SIZE[0], IMAGE_SIZE[1])
    img = cv.resize(img, resize)
    return img

def one_hot_encoding(class_number):
    ha = [0,0,0,0]
    ha[class_number] = 1
    return ha

In [ ]:
## TRAIN SET
# concatenate
train_concat = pd.concat([shuffled_mild_train, shuffled_moderate_train, shuffled_non_train, shuffled_verymild_train])
# shuffle
train_concat = train_concat.sample(frac=1)
# load image
train_concat["X"] = train_concat["X"].apply(load_img)

y_train_lab = train_concat["y"]
y_train_lab = np.array(y_train_lab) # convert into a numpy array

# apply one-hot encoding
train_concat["y"] = train_concat["y"].apply(one_hot_encoding)

# stack images and labels
xtrain = np.stack(train_concat["X"])
ytrain = np.stack(train_concat["y"])


## VALIDATION SET
val_concat = pd.concat([shuffled_mild_val, shuffled_moderate_val, shuffled_non_val, shuffled_verymild_val])
val_concat = val_concat.sample(frac=1)
val_concat["X"] = val_concat["X"].apply(load_img)
val_concat["y"] = val_concat["y"].apply(one_hot_encoding)
xval = np.stack(val_concat["X"])
yval = np.stack(val_concat["y"])

In [ ]:
## TEST SET
test_concat = pd.concat([mild_test, moderate_test, non_test, verymild_test])
test_concat = test_concat.sample(frac=1)
test_concat["X"] = test_concat["X"].apply(load_img)
test_concat["y"] = test_concat["y"].apply(one_hot_encoding)
xtest = np.stack(val_concat["X"])
ytest = np.stack(val_concat["y"])

In [ ]:
# plot some images per class

ncols = 10

fig, axs = plt.subplots(nrows=4, ncols=ncols, figsize=(20, 10))

for fname,ax in zip(shuffled_non.loc[:ncols, "X"], axs[0,:]):
    im = load_img(fname)
    ax.imshow(im)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)

for fname,ax in zip(shuffled_verymild.loc[:ncols, "X"], axs[1,:]):
    im = load_img(fname)
    ax.imshow(im)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)
    
for fname,ax in zip(shuffled_mild.loc[:ncols, "X"], axs[2,:]):
    im = load_img(fname)
    ax.imshow(im)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)

for fname,ax in zip(shuffled_moderate.loc[:ncols, "X"], axs[3,:]):
    im = load_img(fname)
    ax.imshow(im)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)

axs[0,0].set_ylabel("Non Dementia")
axs[1,0].set_ylabel("Very Mild Dementia")
axs[2,0].set_ylabel("Mild Dementia")
axs[3,0].set_ylabel("Moderate Dementia")

In [ ]:
# resnet50
base_model = ResNet50(input_shape=(224,224,3), 
                   include_top=False,
                   weights="imagenet")

In [ ]:
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
data = (xtrain, ytrain, xtest, ytest)

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("alzheimer_model.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.5, verbose=1, mode="auto",
                              cooldown=5, patience=10, min_lr=0.00001)

# Adding class weights to compensate class imabalance
# class_weights = { 0: 1,
#                   1: 1.5,
#                   2: 3,
#                   3: 20
# }

In [ ]:
def build_model(image_shape, **kwargs):

    model = Sequential()

    model.add(ResNet50(input_shape=(224,224,3), 
                   include_top=False,
                   weights="imagenet"))
    
    model.add(Dropout(0.2))
    
    for _ in range(kwargs['conv_layers']):
        model.add(SeparableConv2D(kwargs['kernels'], kernel_size=(kwargs['kernel_size'],kwargs['kernel_size']), activation='relu', padding='same'))
        model.add(SeparableConv2D(kwargs['kernels'], kernel_size=(kwargs['kernel_size'],kwargs['kernel_size']), activation='relu', padding='same'))
        model.add(BatchNormalization())
        if kwargs['maxpooling'] == 1:
          model.add(MaxPooling2D(pool_size=(1)))
        if kwargs['dropout_cnn']  == 1:
          model.add(Dropout(kwargs['dropout_perc_cnn']))
    
    model.add(AveragePooling2D(pool_size=1))
    model.add(Flatten())
    
    for _ in range(kwargs['layers']):
        model.add(Dense(kwargs['neurons'], activation='relu'))
        model.add(BatchNormalization())
        if kwargs['dropout']  == 1:
          model.add(Dropout(kwargs['dropout_perc_cnn']))
    
    model.add(Dense(4, activation='softmax'))

    return model

In [ ]:
def fit_params(image_shape, data, **kwargs):

    for k in kwargs.keys():
        if 'perc' in k:
            continue
        kwargs[k]=kwargs[k].astype(np.int64)
      
    model = build_model(image_shape, **kwargs)
    
    # categorical crossentropy for loss and Adam for optimiser
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

    if not os.path.exists(f'models/{image_shape[0]}'):
        os.makedirs(f'models/{image_shape[0]}')
    
    callbacks = [checkpoint_cb, early_stopping_cb, reduce_lr]
    
    model.fit(xtrain, ytrain, validation_data=(xval, yval), batch_size=kwargs['batch_size'], epochs=20, verbose=0, callbacks=callbacks)
    
    score = model.evaluate(xtest, ytest)

    model.save(f'models/{image_shape[0]}/{score[1]}.hdf5')

    return score[1]



from functools import partial


fit_with_partial = partial(fit_params, IMAGE_SIZE, data)

In [ ]:
image_shape=(224, 224, 3)


fit_with_partial = partial(fit_params, image_shape, data)

# Boundary definitions
pbounds = {'layers':(1,9), 'neurons':(50, 1000), 'batch_size':(16, 64), 'dropout': (0,1), 'dropout_perc': (0.1,0.5), 
          'dropout_perc_cnn':(0.3, 1.0), 'dropout_cnn':(0,1), 'maxpooling': (0,1), 'conv_layers':(1,6), 
           'kernel_size':(1, 5), 'kernels':(32, 2048)}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2, 
    random_state=1,
)

optimizer.maximize()

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)